In [152]:
import csv
import glob
from pprint import pprint 
import matplotlib.pyplot as plt
from collections import defaultdict

"""read csv file and put in list"""
def turnstile_read_csv(csv_files):
    dict_MTA= defaultdict(list)
    MTA = [] 
    for filename in glob.glob(csv_files):
        file = open(filename, 'r')
        reader = csv.reader(file)
        header = reader.next()
        MTA = list(reader)
        for turnstile in MTA:
            turnstile[-1].strip()
            turnstile_key = tuple(turnstile[0:5])
            dict_MTA[turnstile_key].append(turnstile[5:])
        file.close()
    return dict_MTA


#print({k: dict_sample[k] for k in dict_sample.keys()[:2]})

#for data_file in glob.glob("turnstile_*.txt"):
#    weekly_data_dicts.append(turnstile_read_csv(data_file))

#print({k: dict_MTA[k] for k in dict_MTA.keys()[:2]})
#for filename in glob.glob("turnstile_*.txt"):
 #   print(filename)
#[turnstile_read_csv(csvfile) for csvfile in glob.glob("turnstile_*.txt")]

#data_files = glob.glob("turnstile_*'txt")
#weekly_data-dicts = map(turnstile_read_csv, data_files)


In [153]:
dict_MTA = turnstile_read_csv("turnstile_*.txt")
pprint ({k: dict_MTA[k][:2] for k in dict_MTA.keys()[:5]})

{('JFK03', 'R536', '00-00-02', 'JFK JAMAICA CT1', 'E'): [['IND',
                                                          '06/04/2016',
                                                          '01:00:00',
                                                          'REGULAR',
                                                          '0000087367',
                                                          '0000079390                                       '],
                                                         ['IND',
                                                          '06/04/2016',
                                                          '05:00:00',
                                                          'REGULAR',
                                                          '0000087372',
                                                          '0000079392                                       ']],
 ('N218', 'R112', '01-05-00', 'FORDHAM RD', 'BD'): [['IND',
                   

In [154]:

"""from datetime import datetime, date
from dateutil import parser
def time_series(dict_MTA):
    dict_MTA_ts = {}
    for key in dict_MTA.keys():
        dict_MTA_ts[key] = []
        for i in range(0, len(dict_MTA[key])):
            time = dict_MTA[key][i][2] + " " + dict_MTA[key][i][3]
            four_hr_ct = int(dict_MTA[key][i][5])- int(dict_MTA[key][i-1][5])
            #print "Count: %d" %four_hr_ct
            if four_hr_ct < 0 or four_hr_ct > 4*60*20:
                dict_MTA_ts[key].append([parser.parse(time), None])
            else:
                dict_MTA_ts[key].append([parser.parse(time), four_hr_ct])
    return dict_MTA_ts"""

from datetime import datetime, date
from dateutil import parser
def time_series(dict_MTA):
    dict_MTA_ts = {}
    for key in dict_MTA.keys():
        dict_MTA_ts[key] = []
        for i in range(0, len(dict_MTA[key])):
            time = dict_MTA[key][i][1] + " " + dict_MTA[key][i][2]
            four_hr_ct = int(dict_MTA[key][i][4])- int(dict_MTA[key][i-1][4])
            #print "Count: %d" %four_hr_ct
            if four_hr_ct < 0 or four_hr_ct > 4*60*20:
                dict_MTA_ts[key].append([parser.parse(time), None])
            else:
                dict_MTA_ts[key].append([parser.parse(time), four_hr_ct])
    return dict_MTA_ts


In [192]:
small = dict((k, dict_MTA[k]) for k in (('N134', 'R385', '00-00-00', 'ROCKAWAY BLVD', 'A'), ('A006', 'R079' , '00-00-01' , '5 AV/59 ST','NQR')))
ts_small = time_series(small)
print ts_small[('N134', 'R385', '00-00-00', 'ROCKAWAY BLVD','A')][:10]
print ts_small[('A006', 'R079' , '00-00-01' , '5 AV/59 ST','NQR')][:10]

[[datetime.datetime(2016, 6, 4, 1, 0), None], [datetime.datetime(2016, 6, 4, 5, 0), 19], [datetime.datetime(2016, 6, 4, 9, 0), 186], [datetime.datetime(2016, 6, 4, 13, 0), 166], [datetime.datetime(2016, 6, 4, 17, 0), 197], [datetime.datetime(2016, 6, 4, 21, 0), 176], [datetime.datetime(2016, 6, 5, 1, 0), 102], [datetime.datetime(2016, 6, 5, 5, 0), 23], [datetime.datetime(2016, 6, 5, 9, 0), 105], [datetime.datetime(2016, 6, 5, 13, 0), 129]]
[[datetime.datetime(2016, 6, 4, 2, 0), None], [datetime.datetime(2016, 6, 4, 6, 0), 11], [datetime.datetime(2016, 6, 4, 10, 0), 32], [datetime.datetime(2016, 6, 4, 14, 0), 157], [datetime.datetime(2016, 6, 4, 18, 0), 365], [datetime.datetime(2016, 6, 4, 22, 0), 443], [datetime.datetime(2016, 6, 5, 2, 0), 88], [datetime.datetime(2016, 6, 5, 6, 0), 8], [datetime.datetime(2016, 6, 5, 10, 0), 41], [datetime.datetime(2016, 6, 5, 14, 0), 127]]


In [156]:
dict_MTA_ts = time_series(dict_MTA)

**Combine stations and lines and attempt to adjust for incongruent line-station combos**

In [157]:
def convert(station, line):
    try:
        if station == "34 ST-PENN STA": return "123ACE"
        return {'ACJZ2345': '2345ACJZ', 'ACENQRS1237': '1237ACENQRS','AC1' : '1AC', 
         'JZ456' : '456JZ', 'BDNQR2345' : '2345BDNQR', 'ABCD1' : '1ABCD', 
         'R2345' : '2345R', 'LNQR456' : '456LNQR', 'BD4' : '4BD',
         'ACENGRS1237': '1237ACENQRS'}[line]
    except:
        return line

In [239]:
def combine_into_stations(MTA):
    dict_station = defaultdict(list)
    for key in MTA:
        station_key = key[3] + " _ " + key[4]#convert(key[3], key[4])
        #print station_key
        for interval in MTA[key]:
            #Eprint "Interval: ",
            #print interval 
            #print 14 >= interval[0].hour and interval[0].hour >= 8
            if(14 >= interval[0].hour and interval[0].hour >= 8):
                dict_station[station_key].append([interval[0].weekday(), interval[0].hour, interval[1]])
            #print dict_station
    return dict_station

In [241]:
station_small = combine_into_stations(ts_small)
print station_small

defaultdict(<type 'list'>, {'5 AV/59 ST _ NQR': [[5, 10, 32], [5, 14, 157], [6, 10, 41], [6, 14, 127], [0, 10, 64], [0, 14, 237], [1, 10, 55], [1, 14, 259], [2, 10, 71], [2, 14, 269], [3, 10, 45], [3, 14, 234], [4, 10, 65], [4, 14, 265], [5, 10, 27], [5, 14, 149], [6, 10, 27], [6, 14, 59], [0, 10, 52], [0, 14, 207], [1, 10, 57], [1, 14, 222], [2, 10, 59], [2, 14, 242], [3, 10, 56], [3, 14, 246], [4, 10, 71], [4, 14, 258], [5, 10, 27], [5, 14, 159], [6, 10, 30], [6, 14, 110], [0, 10, 47], [0, 14, 217], [1, 10, 59], [1, 14, 212], [2, 10, 67], [2, 14, 243], [3, 10, 79], [3, 14, 249], [4, 10, 58], [4, 14, 228], [5, 10, 25], [5, 14, 156], [6, 10, 32], [6, 14, 136], [0, 10, 54], [0, 14, 243], [1, 10, 59], [1, 14, 215], [2, 10, 56], [2, 14, 249], [3, 10, 57], [3, 14, 221], [4, 10, 80], [4, 14, 307]], 'ROCKAWAY BLVD _ A': [[5, 9, 186], [5, 13, 166], [6, 9, 105], [6, 13, 129], [0, 9, 628], [0, 13, 241], [1, 9, 678], [1, 13, 331], [2, 9, 696], [2, 13, 245], [3, 9, 599], [3, 13, 251], [4, 9, 620]

In [249]:
dict_stations = combine_into_stations(dict_MTA_ts)
print dict_stations['SUTTER AV-RUTLD _ 3']


[[5, 8, 0], [5, 12, 0], [6, 8, 0], [6, 12, 0], [0, 8, 924], [0, 12, 875], [1, 8, 940], [1, 12, 1045], [2, 8, 1075], [2, 12, 885], [3, 8, 887], [3, 12, 852], [4, 8, 914], [4, 12, 886], [5, 8, 218], [5, 12, 315], [6, 8, 130], [6, 12, 321], [0, 8, 1007], [0, 12, 1005], [1, 8, 880], [1, 12, 850], [2, 8, 1599], [2, 12, 969], [3, 8, 880], [3, 12, 843], [4, 8, 844], [4, 12, 785], [5, 8, 0], [5, 12, 0], [6, 8, 0], [6, 12, 0], [0, 8, 748], [0, 12, 742], [1, 8, 909], [1, 12, 803], [2, 8, 882], [2, 12, 832], [3, 8, 0], [3, 12, 479], [4, 8, 798], [4, 12, 827], [5, 8, 196], [5, 12, 336], [6, 8, 143], [6, 12, 280], [0, 8, 778], [0, 12, 905], [1, 8, 860], [1, 12, 878], [2, 8, 827], [2, 12, 816], [3, 8, 817], [3, 12, 773], [4, 8, 651], [4, 12, 719], [5, 8, 0], [5, 12, 0], [6, 8, 0], [6, 12, 0], [0, 8, 1379], [0, 12, 1368], [1, 8, 1498], [1, 12, 1347], [2, 8, 1497], [2, 12, 1330], [3, 8, 1291], [3, 12, 1301], [4, 8, 1374], [4, 12, 1235], [5, 8, 380], [5, 12, 657], [6, 8, 257], [6, 12, 490], [0, 8, 1413

In [252]:
x = 875+924+1007+1005+748+742+778+905+1379+1368+1413+1291+1221+1241+1329+1251+932+655+689+619+828+652+761+648
print  x/12

1938


**Creates dictionary of format station:[avg count form Mon AM, for Tues AM, ...]**

In [243]:
def days_of_week(dict_station):
    dict_by_day = defaultdict(list)
    for key in dict_station:
        day_counter = [0,0,0,0,0,0,0]
        dict_by_day[key] = [None,None,None,None,None,None,None]
        #print len(dict_station[key])
        
        for entry in dict_station[key]:
            day_counter[entry[0]]+=1
            #print dict_by_day[key]
            #print entry[2]
            if dict_by_day[key][entry[0]] == None and entry[2] != None:
                dict_by_day[key][entry[0]] = float(entry[2])
            elif entry[2] != None:
                dict_by_day[key][entry[0]] += float(entry[2])
        for i in range(0,7):
            dict_by_day[key][i] = dict_by_day[key][i]/(float(day_counter[i])/2)  
    #print day_counter
    
    return dict_by_day

In [245]:
days_small = days_of_week(station_small)
print days_small

defaultdict(<type 'list'>, {'5 AV/59 ST _ NQR': [280.25, 284.5, 314.0, 296.75, 333.0, 183.0, 140.5], 'ROCKAWAY BLVD _ A': [845.5555555555555, 953.0, 942.75, 866.0, 839.25, 460.75, 269.5]})


In [246]:
dict_by_days = days_of_week(dict_stations)
print len(dict_by_days)
print dict_by_days

472
defaultdict(<type 'list'>, {'ELDER AV _ 6': [673.6666666666666, 696.625, 697.75, 689.7021276595744, 668.9166666666666, 408.25, 270.9166666666667], 'CHAMBERS ST _ ACE23': [223.98290598290598, 219.28755364806867, 228.56896551724137, 263.33673469387753, 236.3838383838384, 80.57692307692308, 54.45918367346939], 'AVENUE U _ BQ': [931.6, 1025.8947368421052, 1086.125, 996.7857142857143, 960.3571428571429, 496.65, 415.55], 'BEACH 67 ST _ A': [319.0, 319.85, 313.75, 299.7, 255.9, 140.05, 108.6], 'WHITLOCK AV _ 6': [143.22222222222223, 162.875, 157.75, 116.66666666666667, 157.91666666666666, 82.0, 62.916666666666664], 'CENTRAL AV _ M': [619.375, 647.625, 646.5625, 644.5625, 608.3125, 193.375, 107.5], 'SHEEPSHEAD BAY _ BQ': [1031.2777777777778, 1061.0555555555557, 1049.9722222222222, 998.3333333333334, 949.2222222222222, 355.1666666666667, 262.47222222222223], 'LONGWOOD AV _ 6': [274.7843137254902, 282.54901960784315, 280.078431372549, 291.8333333333333, 268.8627450980392, 156.8235294117647, 

**Prints out all stations to csv**

In [218]:
#print dict_by_days.keys()
#import csv
with open('stations.csv','w') as file:
    writer = csv.writer(file, lineterminator=',')
    for i in dict_by_days.keys():
        writer.writerow([i])

**Finds top stations (or neighborhoods) for a given day of the week**

In [227]:
list_top = sorted(days_small.iteritems(), key=lambda (k,v):(v[0],k))[:10]
print list_top

[('5 AV/59 ST _ NQR', [280.25, 284.5, 314.0, 296.75, 333.0, 183.0, 140.5]), ('ROCKAWAY BLVD _ A', [845.5555555555555, 953.0, 942.75, 866.0, 839.25, 460.75, 269.5])]


In [231]:
def find_top_by_day(dic, top = 0, day_week):
    list_top = sorted(dic.iteritems(), key=lambda (k,v):(-v[day_week],k))[:top]
    return list_top

def find_top_by_week()

In [247]:
find_top(dict_by_days, 20, 0)

[('SUTTER AV-RUTLD _ 3',
  [1938.4166666666667,
   1978.0,
   1988.9166666666667,
   1894.0,
   1845.5833333333333,
   352.8333333333333,
   270.8333333333333]),
 ('MYRTLE AV _ JMZ',
  [1676.6875, 1725.375, 1749.8125, 1699.6875, 1649.5, 839.1875, 738.3125]),
 ('25 AV _ D',
  [1549.75,
   1576.8333333333333,
   1501.4545454545455,
   1814.1739130434783,
   1463.5833333333333,
   699.5833333333334,
   536.0]),
 ('BAY PKWY _ N',
  [1519.55,
   1868.388888888889,
   2091.6875,
   1805.9444444444443,
   1629.888888888889,
   1239.9375,
   988.25]),
 ('30 AV _ NQ',
  [1516.6666666666667,
   1170.7333333333333,
   1575.0416666666667,
   1530.4166666666667,
   1412.8333333333333,
   615.1666666666666,
   451.1666666666667]),
 ('111 ST _ 7', [1460.65, 1337.4, 1486.95, 1430.4, 1438.95, 904.05, 600.05]),
 ('DYCKMAN ST _ A',
  [1435.75,
   1379.75,
   1497.0,
   1457.0,
   1397.0833333333333,
   637.4166666666666,
   503.0]),
 ('NEWKIRK PLAZA _ BQ',
  [1403.0, 1445.2, 1442.1025641025642, 1406.65, 

In [217]:
%matplotlib inline
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import gaussian_kde
import math
plt.style.use('bmh')

In [ ]:
plt.figure(figsize = (20, 8))
for i in 
plt.hist()